<a href="https://colab.research.google.com/github/vichruth/LANE-DETECTION/blob/main/lane_detection(opencv).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
uploaded=files.upload()

In [ ]:
import cv2
import numpy as np

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [vertices], 255)
    masked = cv2.bitwise_and(img, mask)
    return masked

def draw_lines(img, lines):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 3)

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    height, width = frame.shape[:2]
    roi_vertices = np.array([[
        (0, height),
        (width / 2, height / 2),
        (width, height)
    ]], dtype=np.int32)

    cropped = region_of_interest(edges, roi_vertices[0])
    lines = cv2.HoughLinesP(cropped, 1, np.pi/180, 50, maxLineGap=150)

    line_img = np.zeros_like(frame)
    if lines is not None:
        draw_lines(line_img, lines)

    return cv2.addWeighted(frame, 0.8, line_img, 1, 0)

cap = cv2.VideoCapture("lanes_clip.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    lane_frame = process_frame(frame)
    from google.colab.patches import cv2_imshow
    cv2_imshow(lane_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
